import pandas as pd
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter, OrderedDict

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from collections import OrderedDict

In [5]:
df = pd.read_csv('compas-scores-two-years.csv')

In [6]:
df = df[['event', 'is_violent_recid', 'is_recid', 'priors_count', 'juv_other_count','juv_misd_count', 'juv_fel_count', 'race', 'age_cat', 'sex','score_text']]

In [7]:
def preprocess_compas(df):
    df['age_cat'] = df['age_cat'].map({'Less than 25': 0, '25 - 45': 1, 'Greater than 45': 2}).astype(int)    
    df['score_text'] = df['score_text'].map({'Low': 0, 'Medium': 1, 'High': 2}).astype(int)    
    df['race'] = df['race'].map({'Other': 0, 'African-American': 0, 'Hispanic': 0, 'Native American': 0, 'Asian': 0, 'Caucasian': 1}).astype(int)
    df['sex'] = df['sex'].map({'Male': 1, 'Female': 0}).astype(int)    
    
    df.loc[(df['priors_count'] <= 5), 'priors_count'] = 0
    df.loc[(df['priors_count'] > 5) & (df['priors_count'] <= 15), 'priors_count'] = 1
    df.loc[(df['priors_count'] > 15), 'priors_count'] = 2
    
    df.loc[(df['juv_fel_count'] == 0), 'juv_fel_count'] = 0
    df.loc[(df['juv_fel_count'] == 1), 'juv_fel_count'] = 1
    df.loc[(df['juv_fel_count'] > 1), 'juv_fel_count'] = 2
    
    df.loc[(df['juv_misd_count'] == 0), 'juv_misd_count'] = 0
    df.loc[(df['juv_misd_count'] == 1), 'juv_misd_count'] = 1
    df.loc[(df['juv_misd_count'] > 1), 'juv_misd_count'] = 2
    
    df.loc[(df['juv_other_count'] == 0), 'juv_other_count'] = 0
    df.loc[(df['juv_other_count'] == 1), 'juv_other_count'] = 1
    df.loc[(df['juv_other_count'] > 1), 'juv_other_count'] = 2
    return df

In [8]:
df = preprocess_compas(df)

In [9]:
y = df['is_recid']

In [10]:
print(len(y))

7214


In [11]:
df = df.drop(columns=['is_recid', 'is_violent_recid'])

In [12]:
df.head()

,event,priors_count,juv_other_count,juv_misd_count,juv_fel_count,race,age_cat,sex,score_text
0,0,0,0,0,0,0,2,1,0
1,1,0,0,0,0,0,1,1,0
2,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,0,1,2
4,0,0,0,0,0,0,1,1,0


In [13]:
for column in df.columns:
    unique_values = df[column].unique()
    print(f'Unique values in column "{column}": {unique_values}')

Unique values in column "event": [0 1]
Unique values in column "priors_count": [0 1 2]
Unique values in column "juv_other_count": [0 1 2]
Unique values in column "juv_misd_count": [0 1 2]
Unique values in column "juv_fel_count": [0 2 1]
Unique values in column "race": [0 1]
Unique values in column "age_cat": [2 1 0]
Unique values in column "sex": [1 0]
Unique values in column "score_text": [0 2 1]


In [14]:
def match_pattern(string1, string2):
    if len(string1) == len(string2):
        for i in range(len(string1)):
            if string1[i].isdigit() and string2[i].isdigit():
                if string1[i] != string2[i]:
                    return 0
        else:
            return 1
        return 0

In [15]:

#Return the inverted index matrix for the dataset
def preprocessing(dataset):
    
    #Get cardinalities out of the dataset
    cardinalities = []
#     cardinalities = [[0,1],[0,1],[0,1]]
    for col in dataset.columns:
        cardinalities.append(dataset[col].unique().tolist())

    for cardinality in cardinalities:
        cardinality.sort()
        
    num = 1
    for cardinality in cardinalities:
        num *= len(cardinality)+1
        
    print('Total number of patterns in the dataset: ',num)


    #Get unique value combinations count
    dataset_string = []
#     dataset_string = ['010','001','000','011','001']
    for i in range(len(dataset)):
        dataset_string.append("".join(str(x) for x in dataset.iloc[i].values.tolist()))
        
    counts = OrderedDict()

    for item in dataset_string:
        counts[item] = counts.get(item, 0) + 1

    data_unique_values = list(counts.keys())
    data_value_counts = list(counts.values())
    
    inverted_ind = []
    
    #create inverted index matrix
    for i,cardinality in enumerate(cardinalities):
        for cardinality_val in cardinality:
            new_row = []
            for val in data_unique_values:
                if cardinality_val == int(val[i]):
                    new_row.append(1)
                else:
                    new_row.append(0) 
            inverted_ind.append(new_row)
            
    return inverted_ind, data_value_counts, cardinalities
        

def cov(pattern, dataset):  #tested_OK
    """
    Returns the number of instances in the dataset covered by the given pattern.
    """
    count = 0

    for i in range(len(dataset)):
        res = match_pattern(pattern, "".join(str(x) for x in df.iloc[i].values.tolist()))
        if res == 1:
            count += 1
#     print(count)
    return count


def coverage_optimized(pattern, inverted_ind, data_value_counts, cardinalities):  #invertedindices  #tested_OK for the example given in the research paper

    result_and = [1] * len(inverted_ind[0])
    row_index = 0
    for i,x in enumerate(pattern):
        if x!= 'X':
            
            #find the value x in cardinalities[i]
            index = cardinalities[i].index(int(x))
            row_index += index
            inverted_ind_row = inverted_ind[row_index]
            for j in range(len(inverted_ind_row)):
                result_and[j] = int(inverted_ind_row[j] and result_and[j])
            row_index -= index
            row_index += len(cardinalities[i])
        else:
            row_index += len(cardinalities[i])      

    # DOT Product between the above result and the count array for the datapoints
    coverage = sum([x*y for x, y in zip(data_value_counts, result_and)])
    return coverage
    

def generate_parent_nodes(pattern):  #tested_OK
    """
    Generates all parent nodes of the given pattern by replacing one deterministic
    cell with a wildcard character.
    """
    parents = []
    for i in range(len(pattern)):
        new_string = pattern[:i] + "X" + pattern[i+1:]
        if new_string != pattern:
            parents.append(new_string)
    return parents


def generate_nodes(pattern, cardinalities): #tested_OK
    """
    Generates all nodes on the given pattern and cardinalities based on Rule 1.
    """
    
    """
    TODO: Make cardinalities 2D vector, so that it can be traversed to find the children of a pattern
    """

    # Find the index of the right-most deterministic element in the pattern
    index = len(pattern) - 1
    rm_deter = -1
    while index >= 0:
        if pattern[index] != 'X':
            rm_deter = index
            break
        index -= 1

    candidate_nodes = []
    rm_deter += 1
    if rm_deter >= 0:
        while rm_deter < len(pattern):
            index = rm_deter
            for value in cardinalities[index]:
                candidate_node = pattern
                candidate_node = pattern[:index] + str(value) + pattern[index+1:] 
                candidate_nodes.append(candidate_node)
            rm_deter += 1

    return candidate_nodes


def dominance(pattern, mups):
    #iterate through the mups, find if any of the mups is an ancestor for the pattern p return 0
    # if pattern p is an ancestor for any of the mups return 1
    # else return -1
    
    for m in mups:
        for i,x in enumerate(m):
            if m[i] != pattern[i]:
                if m[i] == 'X' and pattern[i] != 'X':
                    return 0
                elif m[i] != 'X' and pattern[i] == 'X':
                    return 1
    
    return -1

    
def deepdiver(dataset, threshold):
    """
    Finds the maximal uncovered patterns in the dataset.
    """ 
    inverted_index, unique_value_counts, cardinalities = preprocessing(dataset);

    stack = ['XXXXXXXXX']
    maximal_uncovered = []
    while stack:
        pattern = stack.pop()
        uncovered_flag = False
        if dominance(pattern, maximal_uncovered) == 0:
            continue
        elif dominance(pattern, maximal_uncovered) == 1:
            uncovered_flag = False
        else: 
            count = coverage_optimized(pattern, inverted_index, unique_value_counts, cardinalities)
            if count < threshold:
                uncovered_flag = True
        if uncovered_flag:
            stack0 = []
            stack0.append(pattern)
            while stack0:
                pattern0 = stack0.pop()
                parent_nodes = generate_parent_nodes(pattern0)
                for p in parent_nodes:
                    count0 = coverage_optimized(p, inverted_index, unique_value_counts, cardinalities )
                    if count0 < threshold:
                        stack0.append(p)
                        break                   
                maximal_uncovered.append(pattern)
        else:
            stack.extend(generate_nodes(pattern, cardinalities))
            
    print('MUPs are: ', maximal_uncovered)
    return maximal_uncovered


In [16]:
inv, d, cardinalities = preprocessing(df)
print(coverage_optimized('XXXXXXXXX',inv,d,cardinalities))


Total number of patterns in the dataset:  110592


7214


In [17]:
deepdiver(df, 10)

Total number of patterns in the dataset:  110592
MUPs are:  ['XXXXX1202']


['XXXXX1202']

In [18]:
deepdiver(df, 500)

Total number of patterns in the dataset:  110592
MUPs are:  ['XXXXXXX02', 'XXXXXXX01']


['XXXXXXX02', 'XXXXXXX01']

In [19]:
deepdiver(df, 1000)

Total number of patterns in the dataset:  110592
MUPs are:  ['XXXXXXX02', 'XXXXXXX01', 'XXXXXXX00']


['XXXXXXX02', 'XXXXXXX01', 'XXXXXXX00']

In [20]:
def getStringsMatchingWithPattern(pattern, df):
    
    
    #Get unique value combinations count
    dataset_string = []
    for i in range(len(df)):
        dataset_string.append("".join(str(x) for x in df.iloc[i].values.tolist()))
        
    
    strings = []
    flag = True
    for s in dataset_string:
        flag = True
        for i,x in enumerate(s):
            if x != pattern[i]:
                if pattern[i] != 'X':
#                     print('string:',s[i])
#                     print('string:',pattern[i])
                    flag = False
                    
        if flag:
            strings.append(s)
            
    print(len(strings))       
    return strings
        
            

In [21]:
getStringsMatchingWithPattern('XXXXXXX02', df)

190


['000000002',
 '000010002',
 '110000102',
 '100000102',
 '010001102',
 '120100102',
 '000000102',
 '110001102',
 '000000102',
 '000000002',
 '000000102',
 '100000002',
 '100000202',
 '020000202',
 '010010002',
 '111001002',
 '000001002',
 '100001002',
 '000000002',
 '111001102',
 '000001202',
 '100000102',
 '000000102',
 '020000202',
 '000000002',
 '100001002',
 '000001102',
 '000000102',
 '100001102',
 '000000002',
 '010000102',
 '010000102',
 '100000102',
 '110001102',
 '000000102',
 '100000102',
 '100000102',
 '100000002',
 '010000102',
 '110000102',
 '000001202',
 '000000002',
 '001201002',
 '010000102',
 '002000002',
 '100001102',
 '000001102',
 '100000002',
 '110100102',
 '010001102',
 '100000102',
 '000001002',
 '100000202',
 '010000202',
 '100001102',
 '112000102',
 '100010002',
 '000000002',
 '012200102',
 '010000202',
 '010000102',
 '001200002',
 '100000002',
 '100000102',
 '100000102',
 '100000102',
 '100000002',
 '120000102',
 '110000102',
 '010001202',
 '100000102',
 '0000

In [22]:
getStringsMatchingWithPattern('XXXXXX000', df)

87


['000001000',
 '000000000',
 '000000000',
 '100000000',
 '000001000',
 '100000000',
 '000010000',
 '000000000',
 '000001000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000001000',
 '000000000',
 '001001000',
 '000000000',
 '100000000',
 '101000000',
 '000000000',
 '000001000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '100001000',
 '000000000',
 '000000000',
 '000000000',
 '000001000',
 '100001000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '100000000',
 '000000000',
 '000000000',
 '000000000',
 '000001000',
 '100000000',
 '000001000',
 '000001000',
 '000000000',
 '000000000',
 '000000000',
 '100000000',
 '000000000',
 '000001000',
 '000000000',
 '000100000',
 '000000000',
 '000000000',
 '100000000',
 '000000000',
 '100000000',
 '000000000',
 '000001000',
 '000000000',
 '000000000',
 '002001000',
 '100000000',
 '100000000',
 '000001000',
 '000000000',
 '000000000',
 '000000000',
 '101000000',
 '000001000',
 '0000